In [104]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import tflearn
import tensorflow as tf
import random
import json
import string
import unicodedata
import sys
from tflearn.data_utils import to_categorical, pad_sequences

# a table structure to hold the different punctuation used
tbl = dict.fromkeys(i for i in range(sys.maxunicode)
                    if unicodedata.category(chr(i)).startswith('P'))

punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

# method to remove punctuations from sentences.
def remove_punctuation(text):
    no_punct = ""
    for char in text:
        if char not in punctuations:
            no_punct = no_punct + char
    return no_punct

# initialize the stemmer
stemmer = LancasterStemmer()

In [107]:
from io import StringIO
import requests
import json
import pandas as pd

df_data_1 = pd.read_csv(get_object_storage_file_with_credentials_2659b32ff9a04774afc2ee0815088bcf('project1', 'Stream_Twitter.csv'))
print(df_data_1.head())

print(df_data_1.shape)

print(df_data_1.groupby(by=['SPAM'])['SPAM'].count())


   S. No.  SPAM                                    Sound Bite Text
0       1     1  RT @foxyCAMS: Watch live HOT shows join for fr...
1       2     1  Watch live HOT shows, join for free ? camslut....
2       3     1  Watch live HOT shows, join for free ? camslut....
3       4     1  Sexy camgirls are hot to hard free camshow ? b...
4       5     1  RT @twitxgnd: pretty pussy #pussy #prettypussy...
(10814, 3)
SPAM
0    5962
1    4852
Name: SPAM, dtype: int64


In [108]:
labels    = df_data_1['SPAM']
sentences = df_data_1['Sound Bite Text']

words = []
# a list of tuples with words in the sentence and category name
docs = []

categories = list([0,1])

In [109]:
rowNum = 0;

for text in sentences:
    each_sentence = remove_punctuation(text)
    w = nltk.word_tokenize(each_sentence)
    words.extend(w)
    docs.append((w, labels[rowNum]))
    rowNum = rowNum + 1
    
words = [stemmer.stem(w.lower()) for w in words]
words = sorted(list(set(words)))

In [110]:
# create our training data
training = []
output = []
# create an empty array for our output, taking 2 as we only have two categories
output_empty = [0] * 2


for doc in docs:
    # initialize our bag of words(bow) for each document in the list
    bow = []
    # list of tokenized words for the pattern
    token_words = doc[0]
    # stem each word
    token_words = [stemmer.stem(word.lower()) for word in token_words]
    # create our bag of words array
    for w in words:
        bow.append(1) if w in token_words else bow.append(0)
        
    output_row = list(output_empty)
    output_row[categories.index(doc[1])] = 1

    # our training set will contain a the bag of words model and the output row that tells
    # which catefory that bow belongs to.
    training.append([bow, output_row])

In [116]:
# shuffle our features and turn into np.array as tensorflow  takes in numpy array
random.shuffle(training)
training = np.array(training)

# trainX contains the Bag of words and train_y contains the label/ category
train_x = list(training[:, 0])
train_y = list(training[:, 1])

In [117]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=100, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 135199  | total loss: 0.00211 | time: 4.112s
| Adam | epoch: 100 | loss: 0.00211 - acc: 0.9988 -- iter: 10808/10814
Training Step: 135200  | total loss: 0.00190 | time: 4.115s
| Adam | epoch: 100 | loss: 0.00190 - acc: 0.9989 -- iter: 10814/10814
--
INFO:tensorflow:/gpfs/global_fs01/cluster/yp-spark-lon02-env5-0101.bluemix.net/user/sfe5-7fdf5ebe7e8062-7f5d27323682/notebook/work/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [118]:
def get_tf_record(sentence):
    global words
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    # bag of words
    bow = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bow[i] = 1

    return(np.array(bow))

In [140]:

df_data_3 = pd.read_csv(get_object_storage_file_with_credentials_2659b32ff9a04774afc2ee0815088bcf('project1', 'TestDataSet.csv'))
df_data_3.head()


,_id,digest,spam
0,V2Ut8i4ttFeDJHtx,🔹🔸🔹🔸🔹🔸🔹.\n#Snapindia #indianstreetphotography ...,news
1,1bDQZK2ecPXzNGg5O,The only reason you both think we need dairy i...,social
2,1lcZR50umJrhv88@s,Swing by @ketsourinemacarons to cool down with...,news
3,1HjyxJyVN7DMT1yuy,Made Ochazuke (rice and fish dish with tea pou...,TPA
4,WXSuvsvFr2j5SKF@,"You'll lose weight, have more energy, and help...",news


In [170]:
prediction = [0] * 2
test_results = list()
i = 1

for row in df_data_3['digest']:
    prediction = model.predict([get_tf_record(row)])
    test_results.append([prediction[0][0], prediction[0][1], (row.encode('utf-8'))])
    print('processing row no: ', i)
    i = i + 1
    
df = pd.DataFrame(test_results, columns = ['No Spam Prob.', 'Spam Prob.', 'Text'])

processing row no:  1
processing row no:  2
processing row no:  3
processing row no:  4
processing row no:  5
processing row no:  6
processing row no:  7
processing row no:  8
processing row no:  9
processing row no:  10
processing row no:  11
processing row no:  12
processing row no:  13
processing row no:  14
processing row no:  15
processing row no:  16
processing row no:  17
processing row no:  18
processing row no:  19
processing row no:  20
processing row no:  21
processing row no:  22
processing row no:  23
processing row no:  24
processing row no:  25
processing row no:  26
processing row no:  27
processing row no:  28
processing row no:  29
processing row no:  30
processing row no:  31
processing row no:  32
processing row no:  33
processing row no:  34
processing row no:  35
processing row no:  36
processing row no:  37
processing row no:  38
processing row no:  39
processing row no:  40
processing row no:  41
processing row no:  42
processing row no:  43
processing row no:  

In [152]:
from io import BytesIO 
import requests 
import json 
import pandas as pd 

def put_file(credentials, local_file_name): 
    """This functions returns a StringIO object containing the file content from Bluemix Object Storage V3.""" 
    f = open(local_file_name,'r') 
    my_data = f.read() 
    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens']) 
    data = {'auth': {'identity': {'methods': ['password'], 'password': {'user': {'name': credentials['username'],'domain': {'id': credentials['domain_id']}, 'password': credentials['password']}}}}} 
    headers1 = {'Content-Type': 'application/json'} 
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1) 
    resp1_body = resp1.json() 
    for e1 in resp1_body['token']['catalog']: 
        if(e1['type']=='object-store'): 
            for e2 in e1['endpoints']: 
                if(e2['interface']=='public'and e2['region']== credentials['region']):    
                    url2 = ''.join([e2['url'],'/', credentials['container'], '/', local_file_name]) 
    
    s_subject_token = resp1.headers['x-subject-token'] 
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'} 
    resp2 = requests.put(url=url2, headers=headers2, data = my_data ) 
    print (resp2)

In [171]:

df.to_csv('Dataset.csv',index=False)

put_file(credentials_2660 ,'Dataset.csv')

<Response [201]>


In [173]:
prediction = model.predict([get_tf_record('Coca-cola pepsi is so good')])
prediction

array([[9.9977738e-01, 2.2261981e-04]], dtype=float32)